In [2]:
from presentation_utils import *
from DoD import view_4c_analysis_baseline as v4c
import glob
import pprint

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# Config

In [4]:
# directory storing the views
dir_path = "./chembl_result/result4/"
# max size of candidate key
candidate_key_size = 2
# sample rows to present
sample_size = 5
# exploration / exploitation (choose next pair to present among top_k)
top_k = 10
# epsilon-greedy
epsilon = 0.1

max_num_interactions = 100

# Run 4C

In [5]:
# Run 4C
print("Running 4C...")

compatible_groups, contained_groups, complementary_groups, contradictory_groups, all_pair_contr_compl = \
    v4c.main(dir_path, candidate_key_size)

print("Compatible groups:")
for group in compatible_groups:
    print(group)

print("Contained groups:")
for group in contained_groups:
    print(group)

print("Contradictory groups:")
for path1, candidate_key_tuple, key_value_tuples, path2 in contradictory_groups:
    print(str(list(candidate_key_tuple)) + ": " + path1 + " - " + path2)

print("Complementary groups:")
for path1, path2, candidate_key_tuple, _, _ in complementary_groups:
    print(str(list(candidate_key_tuple)) + ": " + path1 + " - " + path2)

Running 4C...
Found 11 valid tables
View candidates classify into 5 groups based on schema

Num elements with schema -810468692286326092 is: 4


100%|██████████| 6/6 [00:00<00:00, 22.09it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/6 [00:00<?, ?it/s]/Users/zhiruzhu/Desktop/research/aurum-dod-staging/DoD/view_4c_analysis_baseline.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection1["dummy_col"] = ""
/Users/zhiruzhu/Desktop/research/aurum-dod-staging/DoD/view_4c_analysis_baseline.py:274: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection2["dummy_col"] = ""
 50%|█████     | 3/6 [00:00<00:00, 23.54it/s

Num elements with schema 7289721400757619846 is: 1
Num elements with schema -3609427463666520069 is: 1
Num elements with schema 6090849310395091736 is: 1
Num elements with schema 388403398076202018 is: 4


100%|██████████| 6/6 [00:00<00:00, 24.98it/s]

Compatible groups:
Contained groups:
Contradictory groups:
['compound_name']: ./chembl_result/result4/view_9.csv - ./chembl_result/result4/view_7.csv
['compound_name']: ./chembl_result/result4/view_1.csv - ./chembl_result/result4/view_9.csv
Complementary groups:
['assay_organism', 'compound_name']: ./chembl_result/result4/view_9.csv - ./chembl_result/result4/view_7.csv
['compound_name']: ./chembl_result/result4/view_10.csv - ./chembl_result/result4/view_7.csv
['assay_organism', 'compound_name']: ./chembl_result/result4/view_10.csv - ./chembl_result/result4/view_7.csv
['compound_name']: ./chembl_result/result4/view_10.csv - ./chembl_result/result4/view_9.csv
['assay_organism', 'compound_name']: ./chembl_result/result4/view_10.csv - ./chembl_result/result4/view_9.csv
['compound_name']: ./chembl_result/result4/view_1.csv - ./chembl_result/result4/view_7.csv
['assay_organism', 'compound_name']: ./chembl_result/result4/view_1.csv - ./chembl_result/result4/view_7.csv
['assay_organism', 'comp

# Pruning 4C views
## Remove identical views and keep the contained view with the largest cardinality

In [6]:
view_files = glob.glob(dir_path + "/view_*")
print("Number of views: ", len(view_files))

view_files = prune_compatible_views(view_files, compatible_groups)
print("After pruning compatible views: ", len(view_files))

view_files = prune_contained_views(view_files, contained_groups)
print("After pruning contained views: ", len(view_files))

Number of views:  13
After pruning compatible views:  13
After pruning contained views:  13


# Pre-processing and generating sample rows to present

In [7]:
print("Pre-processing...")

contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict = preprocess(view_files, all_pair_contr_compl, sample_size)

 17%|█▋        | 2/12 [00:00<00:00, 15.37it/s]

Pre-processing...


100%|██████████| 12/12 [00:00<00:00, 21.43it/s]


# Actual presentation

In [9]:
%gui asyncio
# Using asynchronous widgets. Requires ipykernel 4.7 or later
# pip install ipython ipykernel --upgrade

# async
task = present_async(view_files, contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict,
                     top_k, epsilon, max_num_interactions, sample_size)

# sync
# final_view_scores, num_interactions = present(view_files, contr_or_compl_view_pairs, non_contr_or_compl_views, row_to_path_dict,
#                                               top_k, epsilon, max_num_interactions, sample_size)

Output()

# Final view scores

In [9]:
# async
await task
final_view_scores, num_interactions = task.result()

print("Final view scores:")
pprint.pprint(final_view_scores)

print("Number of interactions = " + str(num_interactions))

Final view scores:
[('./toytest/view_158', 10),
 ('./toytest/view_114', 7),
 ('./toytest/view_88', 6),
 ('./toytest/view_126', 6),
 ('./toytest/view_44', 4),
 ('./toytest/view_11', 4),
 ('./toytest/view_215', 0),
 ('./toytest/view_188', 0)]
Number of interactions = 6
